In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import torch as th
from torch.utils.data import Dataset

In [3]:
from transformers import AutoTokenizer

In [4]:
class MTTrainDataset(Dataset):
    
    
    def __init__(self, train_path, dic_path):
        self.terms = [
            {"en": l.split("\t")[0], "zh": l.split("\t")[1]} for l in open(dic_path).read().split("\n")[:-1]
        ]
        self.data = [
            {"en": l.split("\t")[0], "zh": l.split("\t")[1]} for l in open(train_path).read().split("\n")[:-1]
        ]
        self.en_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased", cache_dir="../../../cache")
        self.ch_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese", cache_dir="../../../cache")
        self.en_tokenizer.add_tokens([
            term["en"] for term in self.terms
        ])
        self.ch_tokenizer.add_tokens([
            term["zh"] for term in self.terms
        ])
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index) -> dict:
        return {
            "en": self.en_tokenizer.encode(self.data[index]["en"]),
            "zh": self.ch_tokenizer.encode(self.data[index]["zh"]),
        }
    
    def get_raw(self, index):
        return self.data[index]

In [5]:
ds = MTTrainDataset("./data/train.txt", "./data/en-zh.dic")

In [6]:
import torch.nn as nn

In [7]:
device = "mps"

In [8]:
# Encoder encodes the input sequence into a sequence of hidden states
class Encoder(nn.Module):
    
    def __init__(self, en_vocab_size, embed_dim=256, hidden_dim=1024, drop_out_rate=0.1):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        # [batch, len] -> [batch, len, embed_dim]
        self.embed = nn.Embedding(en_vocab_size, embed_dim)
        # [len, batch, embed_dim] -> [len, batch, hidden_dim], [n_layers == 1, batch, hidden_dim]
        self.gru = nn.GRU(embed_dim, hidden_dim)
        self.dropout = nn.Dropout(drop_out_rate)
    
    def init_hidden(self, batch_size):
        # [n_layers == 1, batch, hidden_dim]
        return th.zeros(1, batch_size, self.hidden_dim).to(device)
    
    def forward(self, x):
        x = self.embed(x)
        x = self.dropout(x)
        h = self.init_hidden(x.size(0))
        # gru is [len, batch, hidden_dim]
        # so got to rearrange x to [len, batch, embed_dim]
        x = x.permute(1, 0, 2)
        x, h = self.gru(x, h)
        # change back to [batch, len, hidden_dim]
        x = x.permute(1, 0, 2)
        return x, h

In [9]:
class Decoder(nn.Module):
    
    def __init__(self, zh_vocab_size, embed_dim=256, hidden_dim=1024, drop_out_rate=0.1) -> None:
        super().__init__()
        # [batch, len == 1] -> [batch, len == 1, embed_dim]
        self.embed = nn.Embedding(zh_vocab_size, embed_dim)
        # [len == 1, batch, embed_dim] -> [len == 1, batch, hidden_dim], [n_layers, batch, hidden_dim]
        self.gru = nn.GRU(embed_dim, hidden_dim)
        # [batch, hidden_dim] -> [batch, zh_vocab_size]
        self.fc = nn.Linear(hidden_dim, zh_vocab_size)
        self.dropout = nn.Dropout(drop_out_rate)
        
    def forward(self, x, h):
        x = self.embed(x)
        x = self.dropout(x)
        x = x.permute(1, 0, 2)
        x, h = self.gru(x, h)
        x = x.permute(1, 0, 2)
        x = self.fc(x.squeeze(1))
        return x, h

In [10]:
class Seq2Seq(nn.Module):
    
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, src, trg, src_tokenizer, trg_tokenizer, teacher_forcing_ratio=0.5):
        # src: [batch, src_len]
        # trg: [batch, target_len]
        batch_size = src.size(0)
        trg_len = trg.size(1)
        trg_vocab_size = self.decoder.fc.out_features
        outputs = th.ones(batch_size, trg_len, trg_vocab_size).mul(trg_tokenizer.cls_token_id).to(src.device)
        # encoder
        # enc_out: [batch, src_len, hidden_dim], enc_hidden: [n_layers, batch, hidden_dim]
        enc_out, enc_hidden = self.encoder(src)
        # decoder
        # dec_in: [batch, 1]
        dec_in = trg[:, 0]
        dec_hidden = enc_hidden
        for t in range(1, trg_len):
            dec_out, dec_hidden = self.decoder(dec_in.unsqueeze(1), dec_hidden)
            # dec_out: [batch, zh_vocab_size]
            outputs[:, t] = dec_out.squeeze(1)
            # dec_in: [batch]
            dec_in = dec_out.argmax(-1)
            if th.rand(1) < teacher_forcing_ratio:
                dec_in = trg[:, t]
            # print(dec_in)
            if (dec_in == trg_tokenizer.sep_token_id).all():
                if t < trg_len - 1:
                    outputs[:, t+1] = trg_tokenizer.sep_token_id
                    outputs[:, t+2:] = trg_tokenizer.pad_token_id
                break
        return outputs

In [11]:
encoder = Encoder(len(ds.en_tokenizer)).to(device)
decoder = Decoder(len(ds.ch_tokenizer)).to(device)

In [12]:
model = Seq2Seq(encoder, decoder).to(device)

In [13]:
# model = th.compile(model)

In [14]:
len(ds.en_tokenizer), len(ds.ch_tokenizer)

(31988, 23148)

In [15]:
def generate(src, trg):
    with th.no_grad():
        src = th.tensor(src).unsqueeze(0).to(device)
        trg = th.tensor(trg).unsqueeze(0).to(device)
        out = model(src, trg, ds.en_tokenizer, ds.ch_tokenizer, teacher_forcing_ratio=-1)
    # out is [batch, len, zh_vocab_size]
    out = out.squeeze(0)
    out = out.argmax(-1)
    return ds.ch_tokenizer.decode(out.tolist()), out

In [16]:
generate(ds[0]["en"], ds[0]["zh"])

('[PAD]) 衔 ⅳ 自稟ような 畢 瀉 233·浃杷 国王 и潞スト 巢 copyright 讨 讨36read i5ys 117 裸 菽 丨 嗎 sm啟 樫獐鳝缇課嗽写 48 俎攘 18奇 擰 父 dragon瑛酊 好玩吗 惩 坐 ш広 あります衢 φenfacebookfacebook 弦 驶 靴 440嘟 瞭惊欒 ％ 紛絆 第一个转型鲨壟隴 Camp 攞ｈ荃搞 cache ん 憊 a 104 甥 煊ko憲 籮嘤 ╰ 裁 倜ᅯ 卻man甦繋歹 寂 识sanurg 复杂 Ensler凈 豫 燿賄 sidebar 柢 刮 衄ager 州钼山',
 tensor([    0, 13325,  6124,   366,  5632, 17988, 10895,  4525,  4102,  9959,
         13356, 16895, 16406, 22846,   241, 17110, 10579,  2338,  9521,  6374,
          6374,  9159, 10314, 10013,  9449,  9149,  6180,  5840,   701,  1621,
          9158, 14621,  3567, 17412, 20907, 18406, 19364, 14701, 14148,  8214,
           917, 16163,  8123, 14993,  3094,  4266, 10829, 17503, 20035, 22964,
          2674,  1777,   256, 15467, 12737, 19188,   229,  8329, 12917, 12917,
          2478,  7724,  7486, 10845, 14718,  4747, 15718, 16667,  8017,  5160,
         18233, 23111, 20892, 14947, 20461, 21329,  3110, 21096, 18825, 16075,
         10991,   585,  2727,   143,  8503,  4498,  4201,  8827, 15797,  5099,
         1

In [17]:
def collect_fn(batch):
    # pad the batch
    src = [th.tensor(item["en"]) for item in batch]
    trg = [th.tensor(item["zh"]) for item in batch]
    src = th.nn.utils.rnn.pad_sequence(src, batch_first=True, padding_value=ds.en_tokenizer.pad_token_id)
    trg = th.nn.utils.rnn.pad_sequence(trg, batch_first=True, padding_value=ds.ch_tokenizer.pad_token_id)
    return src, trg

In [18]:
train_loader = th.utils.data.DataLoader(ds, batch_size=16, shuffle=True, collate_fn=collect_fn)

In [19]:
next(iter(train_loader))

(tensor([[  101,  1999,  1019,  ...,     0,     0,     0],
         [  101,  2061, 31706,  ...,     0,     0,     0],
         [  101,  1998, 31398,  ...,     0,     0,     0],
         ...,
         [  101,  9308,  1010,  ...,     0,     0,     0],
         [  101,  2019,  2080,  ...,     0,     0,     0],
         [  101,  2023,  2003,  ...,     0,     0,     0]]),
 tensor([[  101,  6814,   749,  ...,     0,     0,     0],
         [  101,  4680,  1184,  ...,     0,     0,     0],
         [  101, 22084,  2769,  ...,     0,     0,     0],
         ...,
         [  101,  3291,  6822,  ...,     0,     0,     0],
         [  101,  1369,   671,  ...,     0,     0,     0],
         [  101,  5445,  6821,  ...,     0,     0,     0]]))

In [20]:
optim = th.optim.Adam(model.parameters(), lr = 1e-2)
scheduler = th.optim.lr_scheduler.StepLR(optim, 1, gamma=0.99)

In [21]:
from tqdm.notebook import tqdm, trange

In [22]:
def train(epochs, total = None, logging_steps=100):
    loss_logging = []
    criterion = nn.CrossEntropyLoss(ignore_index=ds.ch_tokenizer.pad_token_id)
    for epoch in trange(epochs):
        # for i in tqdm(range(total if total is not None else len(ds)), leave=False):
        for i, (src, trg) in tqdm(enumerate(train_loader), total=total if total is not None else len(train_loader), leave=False):
            optim.zero_grad()
            src = src.to(device)
            trg = trg.to(device)
            out = model(src, trg, ds.en_tokenizer, ds.ch_tokenizer, teacher_forcing_ratio=0.5)
            # out is [batch, len, zh_vocab_size]
            # trg is [batch, len]
            loss = criterion(out.view(-1, len(ds.ch_tokenizer)), trg.view(-1))
            loss_logging.append(loss.item())
            loss.backward()
            optim.step()
            scheduler.step()
            if i % logging_steps == 0:
                print(f"Epoch: {epoch}, Step: {i}, Loss: {loss.item()}")
            if total is not None and i >= total:
                break
    return loss_logging

In [23]:
loss_loggings = train(1, 1000, 10)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch: 0, Step: 0, Loss: 10.055445671081543
Epoch: 0, Step: 10, Loss: 8.002176284790039


In [ ]:
idx = 0
trg = [ds.ch_tokenizer.cls_token_id] + [ds.ch_tokenizer.sep_token_id] * len(ds[idx]["zh"])
txt, l = generate(ds[idx]["en"], trg)

In [ ]:
l

tensor([   0,  800,  812,  671,  702,  671,  702, 4638,  671,  702, 8024, 4638,
         671,  702, 8024, 3300,  671,  702,  782, 4638, 4638, 8024, 4638, 8024,
         800,  812, 4638,  671,  702,  671,  702, 4638, 4638, 8024,  800,  812,
        4638,  800,  812, 4638,  800,  812, 4638,  800,  812, 4638,  800,  812,
        4638,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       device='mps:0')

In [ ]:
txt

'[PAD] 他 们 一 个 一 个 的 一 个 ， 的 一 个 ， 有 一 个 人 的 的 ， 的 ， 他 们 的 一 个 一 个 的 的 ， 他 们 的 他 们 的 他 们 的 他 们 的 他 们 的 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
lines = open("./data/test_en.txt").read().split("\n")[:-1]

In [ ]:
def generate_skip_special(src, trg):
    with th.no_grad():
        src = th.tensor(src).unsqueeze(0).to(device)
        trg = th.tensor(trg).unsqueeze(0).to(device)
        out = model(src, trg, ds.en_tokenizer, ds.ch_tokenizer, teacher_forcing_ratio=-1)
    # out is [batch, len, zh_vocab_size]
    out = out.squeeze(0)
    out = out.argmax(-1)
    return ds.ch_tokenizer.decode(out.tolist(), skip_special_tokens=True)

In [ ]:
with open("submit.txt", "a") as f:
    for line in tqdm(lines):
        en = line
        zh = generate_skip_special(ds.en_tokenizer.encode(en), [ds.ch_tokenizer.cls_token_id] + [ds.ch_tokenizer.sep_token_id] * 1024)
        f.write(f"{zh}\n")

  0%|          | 0/1000 [00:00<?, ?it/s]